<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/auto_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [132]:
pip install pyxlsb

In [174]:
#actualização de nomes de ficheiros e filtros

activacao_negativa="Ativações_Negativas_fev24_em_mar24_ONE.xlsb"

cadastro="Cadastro.xlsb"

capitalizacao="02.2024 - Capitalização_Eq. Interna Obv.xlsb"

filtro_cap="Activação"

In [175]:
#carregar e definir ficheiro de activações negativas

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()


activacao_negativa_df=pd.read_excel(activacao_negativa, sheet_name=0)
cadastro_df=pd.read_excel(cadastro, sheet_name=0)
capitalizacao_df=pd.read_excel(capitalizacao, sheet_name="act")

Saving Ativações_Negativas_fev24_em_mar24_ONE.xlsb to Ativações_Negativas_fev24_em_mar24_ONE (4).xlsb


Saving Cadastro.xlsb to Cadastro (4).xlsb


In [177]:
#função para criar a coluna cav e adicioná-la à bd de activações negativas

extensao_total="40168941189470163540"

def extractor (str1):

  dicionario_extensao={"entidade": 6, "delegacao": (6,12), "cav": 12}

  for i, j in dicionario_extensao.items():
    if i=="delegacao":
      start_index, end_index=j
      delegacao=str1[start_index:end_index]
    elif i=="cav":
      start_index=j
      cav=str1[start_index:]
    else:
      entidade=str1[:j]
  return cav


activacao_negativa_df["cav"]=activacao_negativa_df["CONCATENATE"].apply(extractor).astype("int")



activacao_negativa_df["cav"].head()


0    70163540
1    70163540
2    80077863
3    80130177
4    80123513
Name: cav, dtype: int64

In [178]:
#eliminar duplicados no ficheiro do cadastro

cadastro_short=cadastro_df[["Cav outbound", "User Aplicações"]].drop_duplicates()


In [181]:
#criar a coluna username na listagem de activações negativas


#converter cav outbound numa coluna "string"

#cadastro_short["Cav outbound"]=cadastro_short["Cav outbound"].astype("str")

activacao_negativa_df2= \
activacao_negativa_df.merge(cadastro_short[["Cav outbound", "User Aplicações"]], right_on="Cav outbound", left_on="cav", how="left")

activacao_negativa_df2.head()


print(f"existem {activacao_negativa_df2['User Aplicações'].isna().sum()} valores nulos na dataframe")

print(activacao_negativa_df2.shape)

activacao_negativa_df2.drop("Cav outbound", axis=1) #eliminar coluna "Cav outbound" que vem do cadastro

activacao_negativa_df2.loc[activacao_negativa_df2["cav"].isna()==True, ["cav", "Cav outbound"]]

existem 4 valores nulos na dataframe
(131, 110)


,cav,Cav outbound
32,70135131,NaN
33,70135131,NaN
72,80153411,NaN
114,80166426,NaN


In [165]:
#filtrar capitalização por linhas de activação negativa

capitalizacao_df2=capitalizacao_df.loc[capitalizacao_df["Tipo.1"]==filtro_cap, ["Order Item", "valor unit pelos pontos Op", \
                    "valor unit pelos pontos Sup", "valor unit pelos pontos sup tr", \
                    "valor unit pelos pontos Co"]] #Tipo.1 é o nome da coluna que tem o tipo de movimento

#criar colunas que vêm da capitalização: quantidade op, quantidade sup, quantidade tc, quantidade coord, quantidade coord2

activacao_negativa_df3=activacao_negativa_df2.merge(capitalizacao_df2, on="Order Item", how="left")

activacao_negativa_df3.shape



(131, 114)

In [170]:
#multiplicar os valores das activações por -1 (porque são negativas)

activacao_negativa_df3=\
activacao_negativa_df3.loc[:, ["valor unit pelos pontos Op", "valor unit pelos pontos Sup", "valor unit pelos pontos sup tr", \
                           "valor unit pelos pontos Co"]].apply(lambda j: j*-1)

In [166]:
activacao_negativa_df3.to_excel("output.xlsx", index=False)

In [167]:
#fazer download do ficheiro

files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>